# DQLab Data Champion 3 
#### Author: Yevonnael Andrew
#### E-Mail: yevonnael.andrew@gmail.com

In [1]:
import pymysql # untuk mengambil data dari server
import pandas as pd

In [2]:
con = pymysql.connect(user='guest',
                    password = 'relational',
                    host='relational.fit.cvut.cz',
                    port=3306,
                    db='UW_std')
cur = con.cursor(pymysql.cursors.DictCursor)

In [3]:
cur.execute("SHOW TABLES") #untuk melihat tabel apa saja yang tersedia dalam database
tables = [c for c in cur]
tables

[{'Tables_in_UW_std': 'advisedBy'},
 {'Tables_in_UW_std': 'course'},
 {'Tables_in_UW_std': 'person'},
 {'Tables_in_UW_std': 'taughtBy'}]

In [4]:
cur.execute('SELECT * FROM course')
course = pd.DataFrame(cur.fetchall())
course.head()

,course_id,courseLevel
0,5,Level_300
1,11,Level_300
2,18,Level_300
3,104,Level_300
4,124,Level_300


In [5]:
course.to_csv(r'course.csv'); print("File berhasil disimpan dengan nama course.csv")

File berhasil disimpan dengan nama course.csv


In [6]:
cur.execute('SELECT * FROM advisedBy')
advisedBy = pd.DataFrame(cur.fetchall())
advisedBy.head()

,p_id,p_id_dummy
0,96,5
1,118,5
2,183,5
3,263,5
4,362,5


In [7]:
advisedBy.to_csv(r'advisedBy.csv'); print("File berhasil disimpan dengan nama advisedBy.csv")

File berhasil disimpan dengan nama advisedBy.csv


In [8]:
cur.execute('SELECT * FROM person')
person = pd.DataFrame(cur.fetchall())
person.head()

,p_id,professor,student,hasPosition,inPhase,yearsInProgram
0,3,0,1,0,0,0
1,4,0,1,0,0,0
2,5,1,0,Faculty,0,0
3,6,0,1,0,Post_Quals,Year_2
4,7,1,0,Faculty_adj,0,0


In [9]:
person.to_csv(r'person.csv'); print("File berhasil disimpan dengan nama person.csv")

File berhasil disimpan dengan nama person.csv


In [10]:
cur.execute('SELECT * FROM taughtBy')
taughtBy = pd.DataFrame(cur.fetchall())
taughtBy.head()

,course_id,p_id
0,0,40
1,1,40
2,2,180
3,3,279
4,4,107


In [11]:
taughtBy.to_csv(r'taughtBy.csv'); print("File berhasil disimpan dengan nama taughtBy.csv")

File berhasil disimpan dengan nama taughtBy.csv


In [12]:
con.close() # Memutuskan sambungan dengan remote database

**Memulai proses penyatuan variabel menjadi satu file csv**

Database ini bersifat relasional, yang artinya:
1. Satu tabel (variabel) dengan tabel lainnya memiliki suatu 'key' yang identik
2. Key ini yang menjelaskan relasi dari tabel tersebut dengan tabel lainnya

<img src="https://relational.fit.cvut.cz/assets/img/datasets-generated/UW_std.svg" width="400" height="300" align="left">


Dari gambar di samping, kita bisa melihat bahwa:
1. Tabel *person*, *advisedBy* dan *taughtBy* memiliki 'key' yang sama yaitu p_id
2. Tabel taughtBy memiliki variabel course_id yang dijelaskan oleh tabel *course*

**Kita akan melakukan *FULL OUTER JOIN* untuk memastikan bahwa penggabungan data dilakukan secara utuh sehingga tidak ada data yang terbuang**

**FULL OUTER JOIN** adalah metode yang digunakan untuk menggabungkan semua baris dari dua atau lebih tabel. Dengan gabungan luar penuh, tidak ada satu baris yang dibuang.

In [13]:
# join1: Menggabungkan tabel 'person' dengan 'advisedBy'
join1 = pd.merge(person, advisedBy, on='p_id', how='outer')

# join2: Menggabungkan tabel 'taughtBy' dengan 'course'
join2 = pd.merge(taughtBy, course, on='course_id', how ='outer')

# finaljoin: Menintegrasikan join1 dengan join2
finaljoin = pd.merge(join1, join2, on='p_id', how='outer')

In [14]:
finaljoin

,p_id,professor,student,hasPosition,inPhase,yearsInProgram,p_id_dummy,course_id,courseLevel
0,3.0,0,1,0,0,0,NaN,NaN,NaN
1,4.0,0,1,0,0,0,NaN,NaN,NaN
2,5.0,1,0,Faculty,0,0,NaN,19.0,Level_500
3,5.0,1,0,Faculty,0,0,NaN,51.0,Level_400
4,5.0,1,0,Faculty,0,0,NaN,71.0,Level_500
...,...,...,...,...,...,...,...,...,...
439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,131.0,Level_500
440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,135.0,Level_500
441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152.0,Level_500
442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.0,Level_500


In [15]:
print("Hasil dari FULL JOIN dataset ini memiliki dimensi baris dan kolom: {}.".format(finaljoin.shape))

print("Ada {} nilai p_id yang unik (unique value).".format(len(finaljoin['p_id'].unique())))

Hasil dari FULL JOIN dataset ini memiliki dimensi baris dan kolom: (444, 9).
Ada 279 nilai p_id yang unik (unique value).


In [16]:
finaljoin.to_csv(r'FINAL.csv'); print("File hasil olahan berhasil disimpan dengan nama FINAL.CSV")

File hasil olahan berhasil disimpan dengan nama FINAL.CSV


#### The end of the analysis.